## ChatBot

# Import Modules #

In [4]:
import sys
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import random
import requests
import gmaps
from gmaps import Geocoding
from random import randint
import sklearn.model_selection as m_sel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from uber_rides.session import Session
from uber_rides.client import UberRidesClient
import uber_rides.errors as uber_error


session = Session(server_token='gQqt-C_l3O2CSZ0Y2TNjIRJQmSmowvcJPMAOBLoB')
client = UberRidesClient(session)

In [5]:
sys.path

['',
 '/home/user/Project/test/lib/python35.zip',
 '/home/user/Project/test/lib/python3.5',
 '/home/user/Project/test/lib/python3.5/plat-x86_64-linux-gnu',
 '/home/user/Project/test/lib/python3.5/lib-dynload',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '/home/user/Project/test/lib/python3.5/site-packages/lockfile-0.12.2-py3.5.egg',
 '/home/user/Project/test/lib/python3.5/site-packages/joblib-0.11-py3.5.egg',
 '/home/user/Project/test/lib/python3.5/site-packages/nose-1.3.7-py3.5.egg',
 '/home/user/Project/test/lib/python3.5/site-packages',
 '/home/user/Project/Work/skdata/build/py3k',
 '/home/user/Project/test/lib/python3.5/site-packages/skdata-0.0.4-py3.5.egg',
 '/home/user/Project/test/lib/python3.5/site-packages/IPython/extensions',
 '/home/user/.ipython']

# Random No Generation #

In [6]:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

# Brill Tagger #

In [7]:
from nltk.wsd import lesk
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
import tkinter
from nltk.tag import brill, brill_trainer
from nltk.tag.brill_trainer import BrillTaggerTrainer
from nltk.data import load
from nltk.corpus.reader import TaggedCorpusReader

train_data = TaggedCorpusReader('.', 'tagged_input_sentences.txt', sep="/")
traindata= list(train_data.tagged_sents())
postag= load('taggers/maxent_treebank_pos_tagger/english.pickle')
templates = [
        brill.Template(brill.Pos([-1])),
        brill.Template(brill.Pos([1])),
        brill.Template(brill.Pos([-2])),
        brill.Template(brill.Pos([2])),
        brill.Template(brill.Pos([-2, -1])),
        brill.Template(brill.Pos([1, 2])),
        brill.Template(brill.Pos([-3, -2, -1])),
        brill.Template(brill.Pos([1, 2, 3])),
        brill.Template(brill.Pos([-1]), brill.Pos([1])),
        brill.Template(brill.Word([-1])),
        brill.Template(brill.Word([1])),
        brill.Template(brill.Word([-2])),
        brill.Template(brill.Word([2])),
        brill.Template(brill.Word([-2, -1])),
        brill.Template(brill.Word([1, 2])),
        brill.Template(brill.Word([-3, -2, -1])),
        brill.Template(brill.Word([1, 2, 3])),
        brill.Template(brill.Word([-1]), brill.Word([1]))]        
trainer = BrillTaggerTrainer(postag, templates = templates, trace = 3)
brill_tagger = trainer.train(traindata, max_rules = 10)

TBL train (fast) (seqs: 45; tokens: 431; tpls: 18; min score: 2; min acc: None)
Finding initial useful rules...
    Found 974 useful rules.

           B      |
   S   F   r   O  |        Score = Fixed - Broken
   c   i   o   t  |  R     Fixed = num tags changed incorrect -> correct
   o   x   k   h  |  u     Broken = num tags changed correct -> incorrect
   r   e   e   e  |  l     Other = num tags changed incorrect -> incorrect
   e   d   n   r  |  e
------------------+-------------------------------------------------------
  27  32   5   3  | VB->NNP if Pos:TO@[-1]
  13  15   2   1  | NN->NNP if Pos:,@[1]
   8   9   1   5  | JJ->NN if Pos:IN@[-3,-2,-1]
   8   8   0   0  | IN->NN if Word:hsr@[-1]
   6   6   0   0  | RB->NNP if Pos:,@[-1]
   5   5   0   0  | PRP->NN if Pos:VBP@[1]
   4   4   0   0  | IN->NNP if Pos:,@[-1]
   4   4   0   0  | NN->NNP if Pos:,@[1]
   4   4   0   1  | JJ->NNP if Pos:TO@[-3,-2,-1]
   4   4   0   0  | NN->NNP if Word:stackroute@[-2]


# Source and Destination Extraction From Sentence # 

In [8]:
def extract_location(inp):
    tagged = brill_tagger.tag(word_tokenize(inp))
    source = None
    destination = None
    chunkGram = """Source: {<IN>(<NN.*><,>?)+}"""
    chunkParser = nltk.RegexpParser(chunkGram)
    chunked = chunkParser.parse(tagged)
    for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Source'):
        source = ' '.join(list(zip(*subtree))[0][1:]) 
    
    chunkGram = """Destination: {<TO>(<NN.*><,>?)+}"""
    chunkParser = nltk.RegexpParser(chunkGram)
    chunked = chunkParser.parse(tagged)
    for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Destination'):
        destination = ' '.join(list(zip(*subtree))[0][1:]) 
    return source, destination

# Training Intent Mapper #

In [11]:
nb_naive, vect = train_intent_mapper()

In [143]:
v = vect.transform("ShreeVasta")

ValueError: Iterable over raw text documents expected, string object received.

In [142]:
list(v.data)

[]

In [10]:
def train_intent_mapper():
    cab_bot_data_df=pd.read_csv('cab_bot_data.csv')
    cab_bot_data_df['Category_label'] = cab_bot_data_df.Category.map({'Greetings':1, 
                                        'Look':2, 'Book':3, 'Fare_Estimation':4,'Cancel':5, 'Duration':6})
    X = cab_bot_data_df.Questions
    y = cab_bot_data_df.Category_label
    X_train,X_test,Y_train,Y_test = m_sel.train_test_split(X,y,test_size=0.30,random_state=30)
    vect = CountVectorizer()
    vect.fit(X_train)
    X_train_dtm = vect.transform(X_train)
    nb = MultinomialNB()
    nb.fit(X_train_dtm, Y_train)
    return (nb,vect)
    #nb_naive, vect = train_intent_mapper()

# Function For Finding Intent Of a Question #

In [7]:
def find_intent(text,multinomial_nb,vect):
    category = ['Greetings','Look','Book','Fare_Estimation',
                'Cancel','Duration']
    txt_l=[text]
    vect_text = vect.transform(txt_l)
    predict_value = multinomial_nb.predict(vect_text)
    predict_val=int(predict_value)
    intent = category[predict_val-1]
    return intent.lower() 

# Function For Cab Details As Data Frame #

In [8]:
def uber_cab_details(cab_dict):
    columns = ['Car type', 'Availability', 'Distance(Km)', 'Minimum Price', 'Maximum Price', 'ETA(min)' ]
    index=list(range(len(cab_dict['prices'])))
    df = pd.DataFrame(index=index, columns=columns)
    for i in index:
        df.loc[i,'Car type'] = cab_dict['prices'][i]['display_name']
        df.loc[i,'Availability'] = "Yes"
        df.loc[i,'Distance(Km)'] = cab_dict['prices'][i]['distance']
        df.loc[i,'Minimum Price'] = cab_dict['prices'][i]['low_estimate']
        df.loc[i,'Maximum Price'] = cab_dict['prices'][i]['high_estimate']
        df.loc[i,'ETA(min)'] = int(cab_dict['prices'][i]['duration']/100)
    df['Car type']= df['Car type'].str.lower()    
    return df      

# Function For Cab Booking #

In [9]:
def book_uber(source, destination, cabtype, noofseats,response_avail_df, SessInfo):
    df = response_avail_df
    driver_name=["Tharoon Veerasethu","Sharmaji","Chachaji","Chunnu","Munni","beta","daadaji","daadi","bhai","shinde","langoor"]
    for i in range(df.shape[0]):
        x=list(df.loc[i])
        if cabtype in x:
            for j in x:
                if j == cabtype and df.loc[i,'Availability']=='Yes':
                    SessInfo['bookingid']="UBER"+str(random_with_N_digits(6))
                    SessInfo['drivername']=random.choice(driver_name)
                    SessInfo['driverno']=random_with_N_digits(10)
                    SessInfo['carno']="KN-"+str(random_with_N_digits(2))+"-"+str(random_with_N_digits(4))
                    SessInfo['fare']=(df.loc[i,'Minimum Price']+df.loc[i,'Maximum Price'])/2
                    SessInfo['cabtype']=cabtype
                    #SessInfo['noofseats']=noofseats
                    SessInfo['eta']=df.loc[i,'ETA(min)'] 
                    SessInfo['distance']=df.loc[i,'Distance(Km)']
                    SessInfo['booked'] = True
                    break

# Function For Looking Cab #

In [10]:
# when looking for a cab pass seatcount as 1
# when booking a cab pass the exact seat count required
def look_uber(SessInfo, source, destination=None):
    api = Geocoding() 
    try:
        source_loc = api.geocode(source)
        source_lat_long = source_loc[0]['geometry']['location']
        start_lat = source_lat_long['lat']
        start_lng = source_lat_long['lng']
    except:
        print('Sorry the source you have provided is not valid.\n')
        print('Please let me know if you want to know anything more')
        SessInfo = dict()
        return pd.DataFrame({})
    
    
    if destination : 
        try:
            destination_loc = api.geocode(destination)
            destination_lat_long = destination_loc[0]['geometry']['location']
            end_lat=destination_lat_long['lat']
            end_lng=destination_lat_long['lng']

        except:
            print('Sorry the source you have provided is not valid.\n')
            print('Please let me know if you want to know anything more')
            SessInfo = dict()
            return pd.DataFrame({})
        
    else:
        end_lat=12.9173312 # latitude of Central Silk Board
        end_lng=77.6212483 # longitude of Central Silk Board
    try:
        response = client.get_price_estimates(
            start_latitude=start_lat,
            start_longitude=start_lng,
            end_latitude=end_lat,
            end_longitude=end_lng,
            seat_count=2
            )    

        estimate = response.json 
        cab_details_df= uber_cab_details(estimate)
        cab_details_df.fillna(0, axis=1, inplace=True)
        #print('Following are the cabs availability\n')
        #print(cab_details_df)
        return cab_details_df
    except uber_error.ClientError as e:
        print('Sorry please check the location details provided\n')
        print('Please let me know if you want to know anything more')
        SessInfo = dict()
        return pd.DataFrame({})
    

# Storing Information #

In [11]:
def storing_info(inp, intent, SessInfo):
    if intent == 'greetings':
        match1 = re.search('\\bhi|hello|hey\\b', inp)
        match2 = re.search('\\good\smorning|good\sevening|good\snight\\b', inp)
        SessInfo['greet1'] = -1
        SessInfo['greet2'] = -1
        if match1 : SessInfo['greet1'] = match1.group()
        if match2 : SessInfo['greet2'] = match2.group()    
        
    if intent == 'book' or intent == 'look':
    ### look for name entity and of 2 location are ther ethen look for from and 
    ### to and just store the one tagged with gpe
        source , destination = extract_location(inp)
        
        #print('am in storing')
        #print(source, destination)
        
        if source and destination:
            #print('both')
            SessInfo['source'] = source
            SessInfo['destination'] = destination
                        
        elif destination:
            #print('matched field is destination')
            '''check these updates'''
            SessInfo['destination'] = destination
            if 'source' in SessInfo: 
                #print('am here')
                del SessInfo['source']
        elif source:
            #print('matched field is source')
            SessInfo['source'] = source
            '''check these updates'''
            if 'destination' in SessInfo: del SessInfo['destination']
            
        cab_types = ['uberpool','ubergo','uberx', 'uberxl']
        for cab in cab_types:
            if cab in inp:
                SessInfo['cab_type_user'] = cab         

# To Check If Source and Destination Field Is There Or Not #

In [12]:
def handle_require_field(intent):
    ### keys needed for booking 
    if intent== 'book': requiredKey = ['source','destination']
    elif intent == 'look' : requiredKey = ['source']
    if SessInfo['almost_book'] == True : requiredKey.append('cab_type_user')
    
    ### storing missing key require to complete booking
    notAvail = []
    print(requiredKey)
    for key in requiredKey:
        if key not in SessInfo:
            notAvail.append(key)
    ### we can set flag and ask for all the keys simuntaneously
    if(len(notAvail)!=0):
        c = ['We need some details, please provide \n', 'Please provide the following details to proceed:\n', 'Can you give these details\n', 'We require more details, tell us about\n', 'Can you help us with some more details\n']
        print(random.choice(c))
        for key in notAvail:
            if key == 'cab_type_user':
                print(key)
                SessInfo[key] = input() 
                print('No of Seats')
                SessInfo['noofseats']=input()
            else:
                print(key)
                SessInfo[key] = input()

# Confirming Source Destination #

In [13]:
def confirm_src_destination1():
    y = ['Please Wait', 'Almost Done', 'Multiply 23*87 while we work on it', ' Add 323+23+456, lets see who is fast', 'Your Service is almost done', ' Count 5 4 3 2 1' ]
    v = ['Please confirm your source {} {} is correct. Type\'Ok\'', 'Your details are {} {}. Type OK if it is correct', ' Your Source and Destination are {} {}. Type OK to confirm', ' Following are your details {} {}, Type OK to procced', 'Your Booking Source and Destinations are {} {}, Type OK to procced']
    print(random.choice(v).format(SessInfo.get('source',''),'and destination '+SessInfo['destination'] if 'destination' in SessInfo else '' ))
    confirm_inp = input()
    if confirm_inp.lower() == 'ok':
        print(random.choice(y)+'\n')
        if 'destination' in SessInfo: SessInfo['almost_book']= True
    else:
        print('want to update source and destination. Reply with Yes')
        if input()=='yes':
            print('Sorry Provide the info again')
            if 'source' in SessInfo:
                print('source: ')
                SessInfo['source'] = input()
            if 'destination' in SessInfo:
                print('destination: ')
                SessInfo['destination'] = input()
            if 'destination' in SessInfo : SessInfo['almost_book']= True
        else:
            print('Lets have a fresh start')
            if 'destination' in SessInfo: del SessInfo['destination']
            if 'source' in SessInfo: del SessInfo['source']
            return

In [14]:
def confirm_src_destination():
    api = Geocoding()
    try:
        source_loc = api.geocode(SessInfo.get('source',''))
        src_list = (source_loc[0]['formatted_address'].split(',')[:2])
        src = SessInfo.get('source','') + ' ' + ''.join(src_list)
    except:
        return "The source you provided is invalid."
    
    if 'destination' in SessInfo:
        
        try:
            dest_loc = api.geocode(SessInfo.get('destination',''))
            dest_list = (dest_loc[0]['formatted_address'].split(',')[:2])
            dest = ''.join(dest_list)
        except:
            return "The source you provided is invalid."
    
    
    
    v = ['Please confirm your source {} {} is correct. Type\'Ok\'', 'Your details are {} {}. Type OK if it is correct', ' Your Source and Destination are {} {}. Type OK to confirm', ' Following are your details {} {}, Type OK to procced', 'Your Booking Source and Destinations are {} {}, Type OK to procced']
    print(random.choice(v).format(src,'and destination '+ dest if 'destination' in SessInfo else ''))
    confirm_inp = input()
    if confirm_inp.lower() == 'ok':
        if 'destination' in SessInfo: SessInfo['almost_book']= True
    else:
        print('want to update source and destination. Reply with Yes')
        if input()=='yes':
            print('Sorry Provide the info again')
            if 'source' in SessInfo:
                print('source: ')
                SessInfo['source'] = input()
            if 'destination' in SessInfo:
                print('destination: ')
                SessInfo['destination'] = input()
            if 'destination' in SessInfo : SessInfo['almost_book']= True
        else:
            print('Lets have a fresh start')
            if 'destination' in SessInfo: del SessInfo['destination']
            if 'source' in SessInfo: del SessInfo['source']
            return


# Function For Generating Response #

In [16]:
def generating_response(self, inp, intent):
    handle_require_field(intent)
    #y = ['Please Wait', 'Almost Done', 'Multiply 23*87 while we work on it', ' Add 323+23+456, lets see who is fast', 'Your Service is almost done', ' Count 5 4 3 2 1' ]
    resp = ""
    if intent == 'look' :        
        resp = resp + 'Looking for a cab from {} {}'.format(self.SessionInfo.get('source',''),'to '+self.SessionInfo['destination'] if 'destination' in self.SessionInfo else '' )
        resp = resp + '\n' + look_uber(self, self.SessionInfo['source'],self.SessionInfo['destination'])
        ####resp=resp+\n+look_uber()
    
    elif intent == 'book' and self.booked == False:
        
        #### book function           
        resp = resp + 'Looking for a cab from {} {}'.format(self.SessionInfo.get('source',''),'to '+self.SessionInfo['destination'] if 'destination' in self.SessionInfo else '' )
        resp = resp + '\n' + look_uber(self, self.SessionInfo['source'],self.SessionInfo['destination'])
        ####resp=resp+\n+look_uber()
        resp = resp + '\n' + book_uber(self, self.SessionInfo['source'], self.SessionInfo['destination'], self.cab_types, self.SessionInfo.get('noofseats',2))       

    elif intent == 'book' and self.booked == True :
            resp = 'You have already booked a cab with booking id '+ str(self.SessionInfo['bookingid'])
            resp = resp + '\n' + 'We dont provide multiple booking'
        
    '''elif intent == 'duration' :
        x1=['Looking for estimated time of arrival \n',' Please wait! while we look for Estimated Time of arrival of your cab \n','We are looking for estimated time of arrival\n' ]
        if 'bookingid' in SessInfo:
            print(random.choice(x1))
            print('The estimated time of arrival of your cab '+str(SessInfo['cabtype'])+' with booking id '+str(SessInfo['bookingid']) +' is '+str(SessInfo['eta'])+'\n')
        else:
            print("Sorry Sir, you haven't booked any cabs as of now")
            print("Please book a cab inorder to know the expected time of arrival")
            print('Please let me know if you want to know anything more')


    elif intent == 'fare_estimation':
            x2=['Looking for estimated fare \n',' Please wait! while we look for Estimated Fare \n','We are looking for estimated fare\n' ]
            z2= ['Following are your results\n', 'We find something for you\n', ' Have a look at the following options\n']
            if ('estimated' in inp) and ('bookingid' in list(SessInfo.keys())) and ('from' not in inp) and ('to' not in inp):
                print(random.choice(x2))
                print(random.choice(z2))
                print('The estimated fare of your cab '+str(SessInfo['cabtype'])+' with booking id '+str(SessInfo['bookingid']) +' is '+str(SessInfo['fare'])+'\n')
            elif (('estimated' in inp) and ('from' in inp) and ('to' in inp)) or (('from' in inp) and ('to' in inp)):
                src, dest=extract_location(inp)
                df = look_uber(SessInfo,src, dest)
                if not df.empty :
                    print(random.choice(x2))
                    print(random.choice(z2))
                    print("Following are the available cabs and their Minimum and Maximum price")
                    print(df[['Car type','Minimum Price','Maximum Price']])
            else:
                print("Sorry Sir, you haven't booked any cabs as of now")
                print("Please book a cab or give proper source and destination inorder to know the Minimum and Maximum price")
                print('Please let me know if you want to know anything more')        


    elif intent == 'cancel':
        print('Are you sure that you want to cancel the cab Yes/No')
        decision=input().lower()
        if decision == 'yes':
            print('Your cab '+str(SessInfo['cabtype'])+' with booking id '+str(SessInfo['bookingid']) +' is cancelled')
            print("SessInfo before cancelation ",SessInfo)
            SessInfo=None
            print("SessInfo after cancelation ",SessInfo)
        elif decision == 'no':
            print('Your cab is not cancelled')
            print('Please let me know if you want to know anything more')'''
                
        #elif intent == 'schedule':   
    

IndentationError: unexpected indent (<ipython-input-16-3c52119eebe3>, line 4)

# BOT Function #

In [16]:
    
### storing important info
SessInfo = dict() ## refreesh dictionary when user work is completed
SessInfo['almost_book'] = False
SessInfo['booked'] = False
nb_naive, vect = train_intent_mapper()
#already_looked = False
def respond(query):    
    inp = query.lower()
    intent = find_intent(inp, nb_naive, vect)
    storing_info(inp, intent, SessInfo)
    generating_response(inp, intent, SessInfo)            

book a cab from wipro sarjapur
INTENT____________ book
['source', 'destination']
We need some details, please provide 

destination
koramangala
Your details are wipro sarjapur Doddakannelli Sarjapur Road and destination Koramangala Bengaluru. Type OK if it is correct
OK
   Car type Availability  Distance(Km)  Minimum Price  Maximum Price  ETA(min)
0  uberpool          Yes          4.38           70.0           88.0        11
1    ubergo          Yes          4.38          111.0          137.0        11
2     uberx          Yes          4.38          125.0          154.0        11
3    uberxl          Yes          4.38          201.0          247.0        11
['source', 'destination', 'cab_type_user']
Can you help us with some more details

cab_type_user
exit
No of Seats
3
Sorry exit not available. Please provide correct response with booking confirmation, if you want to book 

exit
Thank You


In [179]:
class KabBot:
    
    def __init__(self):
        self.SessionInfo = {
            "source": None,
            "destination": None,
            "intent": None
        }
        self.nb_naive, self.vect = self.train_intent_mapper()
        self.brill_tagger = self.get_brill_tagger()
        
    def get_brill_tagger(self):
        train_data = TaggedCorpusReader('.', 'tagged_input_sentences.txt', sep="/")
        traindata= list(train_data.tagged_sents())
        postag= load('taggers/maxent_treebank_pos_tagger/english.pickle')
        templates = [
                brill.Template(brill.Pos([-1])),
                brill.Template(brill.Pos([1])),
                brill.Template(brill.Pos([-2])),
                brill.Template(brill.Pos([2])),
                brill.Template(brill.Pos([-2, -1])),
                brill.Template(brill.Pos([1, 2])),
                brill.Template(brill.Pos([-3, -2, -1])),
                brill.Template(brill.Pos([1, 2, 3])),
                brill.Template(brill.Pos([-1]), brill.Pos([1])),
                brill.Template(brill.Word([-1])),
                brill.Template(brill.Word([1])),
                brill.Template(brill.Word([-2])),
                brill.Template(brill.Word([2])),
                brill.Template(brill.Word([-2, -1])),
                brill.Template(brill.Word([1, 2])),
                brill.Template(brill.Word([-3, -2, -1])),
                brill.Template(brill.Word([1, 2, 3])),
                brill.Template(brill.Word([-1]), brill.Word([1]))]        
        trainer = BrillTaggerTrainer(postag, templates = templates, trace = 3)
        brill_tagger = trainer.train(traindata, max_rules = 10)
        return brill_tagger
    
    def train_intent_mapper(self):
        cab_bot_data_df = pd.read_csv('cab_bot_data.csv')
        cab_bot_data_df['Category_label'] = cab_bot_data_df.Category.map({'Greetings':1, 
                                            'Look':2, 'Book':3, 'Fare_Estimation':4,'Cancel':5, 'Duration':6})
        X = cab_bot_data_df.Questions
        y = cab_bot_data_df.Category_label
        X_train,X_test,Y_train,Y_test = m_sel.train_test_split(X,y,test_size=0.30,random_state=30)
        vect = CountVectorizer()
        vect.fit(X_train)        
        X_train_dtm = vect.transform(X_train)
        nb = MultinomialNB()
        nb.fit(X_train_dtm, Y_train)
        return (nb, vect)
    
    def find_intent(self, text):
        category = ['Greetings','Look','Book','Fare_Estimation',
                    'Cancel','Duration']
        txt_l = [text]
        vect_text = self.vect.transform(txt_l)
        intent = None
        if len(list(vect_text.data)) != 0:
            predict_value = self.nb_naive.predict(vect_text)
            predict_val = int(predict_value)
            intent = category[predict_val-1].lower()
        return intent
    
    def extract_location(self, inp):
        tagged = self.brill_tagger.tag(word_tokenize(inp))
        chunkGram = """Source: {<IN>(<NN.*><,>?)+}"""
        chunkParser = nltk.RegexpParser(chunkGram)
        chunked = chunkParser.parse(tagged)
        for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Source'):
            self.SessionInfo["source"] = ' '.join(list(zip(*subtree))[0][1:]) 

        chunkGram = """Destination: {<TO>(<NN.*><,>?)+}"""
        chunkParser = nltk.RegexpParser(chunkGram)
        chunked = chunkParser.parse(tagged)
        for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Destination'):
            self.SessionInfo["destination"] = ' '.join(list(zip(*subtree))[0][1:]) 
        
        return self.SessionInfo["source"], self.SessionInfo["destination"]
        
    def storing_info(self, inp, intent):
        if intent == 'book' or intent == 'look':
            self.extract_location(inp)
            cab_types = ['uberpool','ubergo','uberx', 'uberxl']
            self.cab_types = 'ubergo'
            for cab in cab_types: 
                if cab in inp: self.cab_types = inp
        
    def handle_require_field(self, intent):
        requiredKey = ['source','destination']
        notAvail = []
        resp = ''
        boolean = True
        for key in requiredKey:
            if self.SessionInfo[key] == None: 
                notAvail.append(key)
        print(notAvail,len(notAvail))        
        if(len(notAvail)!=0):
            boolean = False
            resp = 'Please provide all the necessary details'
            if(len(notAvail)==2):
                resp = 'You have not provided any source and destination.'
        return resp, boolean
    
    def cab_details(self):
        driver_name = random.choice(["Driver1", "Sharmaji", "Chachaji", "Chunnu", "Munni", "beta", "daadaji", "daadi", "bhai", "shinde"])
        cab_no = 'KA-' + str(random_with_N_digits(2)) + '-AT-' + str(random_with_N_digits(4))
        driver_rating = str(random.choice([3.5, 3.8, 4.0, 4.2, 4.4, 4.5, 4.8, 5]))
        mobile_number = random.choice(['8876','9888','8054','7077']) + str(random_with_N_digits(6))
        return 'Uber driver '+ driver_name + ' with mobile no. '+ mobile_number + ' arriving on cab ' + cab_no + ' has rating of '+ driver_rating + ' stars.'

    def look_uber(self, source, destination, cab_type, intent):
        api = Geocoding()
        try:
            source_loc = api.geocode(source)
            source_lat_long = source_loc[0]['geometry']['location']
            start_lat=source_lat_long['lat']
            start_lng=source_lat_long['lng']
        except:
            return 'Enter Valid Source.\n'

        try:
            destination_loc = api.geocode(destination)
            destination_lat_long = destination_loc[0]['geometry']['location']
            end_lat=destination_lat_long['lat']
            end_lng=destination_lat_long['lng']
        except:
            return 'Enter Valid destination.\n'

        try:
            response = client.get_price_estimates(
                start_latitude=start_lat,
                start_longitude=start_lng,
                end_latitude=end_lat,
                end_longitude=end_lng,
                seat_count=2
                )    

            estimate = response.json 

            for cab_types in estimate['prices']:
                if cab_types['localized_display_name'].lower() == cab_type.lower():
                    if (intent == 'book'):
                        out = 'Booking ' + (cab_type) + ' with averege fare ' + (cab_types['estimate']) + '. Your journy will be ' + str(cab_types['distance']) + ' KM long and will take ' + str(cab_types['duration']/60) + ' minutes. '
                        return out + self.cab_details()
                    else:
                        out = str(cab_type) + ' with averege fare ' + (cab_types['estimate']) + 'is available. Distance will be ' + str(cab_types['distance']) + ' KM and it will take ' + str(cab_types['duration']/60) + ' minutes.'
                        return str(out)
        except uber_error.ClientError as e:
            return 'Distance between two points exceeds 100 miles'
    
    def generating_response(self, inp, intent):
        rep, flag = self.handle_require_field(intent)
        if flag == False:
            return rep
        
        resp = rep
        if intent == 'look' :
            for key in self.SessionInfo:
                print(key, self.SessionInfo[key])
                
            resp = resp + 'Looking for a cab from {}'.format(self.SessionInfo.get('source',''),'to '+ self.SessionInfo['destination'] if self.SessionInfo['destination']!=None else '' )
            resp = resp + '\n' + self.look_uber(self.SessionInfo['source'], self.SessionInfo['destination'], self.cab_types, self.SessionInfo['intent'])
            
        elif intent == 'book':
            resp = resp + 'Looking for a cab from {} {}'.format(self.SessionInfo.get('source',''),'to '+self.SessionInfo['destination'] if self.SessionInfo['destination']!=None else '' )
            #resp = resp + '\n' + self.look_uber(self, self.SessionInfo['source'],self.SessionInfo['destination'])
            resp = resp + '\n' + self.look_uber(self.SessionInfo['source'], self.SessionInfo['destination'], self.cab_types, self.SessionInfo['intent'])       

        '''elif intent == 'book' and self.booked == True :
            resp = 'You have already booked a cab with booking id '+ str(self.SessionInfo['bookingid'])
            resp = resp + '\n' + 'We dont provide multiple booking'
        '''
        return resp 
    
    def respond(self, query):
        inp = query.lower()
        intent = self.find_intent(inp)
        #print(intent)
        self.SessionInfo["intent"] = intent if intent else self.SessionInfo["intent"]
        if intent:
            self.storing_info(inp, intent)
        else: 
            for key in ['source', 'destination']:
                if self.SessionInfo[key] == None:
                    #print(key)
                    self.SessionInfo[key] = inp
        response = self.generating_response(inp, intent)
        return response
        

In [180]:
d = KabBot()

TBL train (fast) (seqs: 45; tokens: 431; tpls: 18; min score: 2; min acc: None)
Finding initial useful rules...
    Found 974 useful rules.

           B      |
   S   F   r   O  |        Score = Fixed - Broken
   c   i   o   t  |  R     Fixed = num tags changed incorrect -> correct
   o   x   k   h  |  u     Broken = num tags changed correct -> incorrect
   r   e   e   e  |  l     Other = num tags changed incorrect -> incorrect
   e   d   n   r  |  e
------------------+-------------------------------------------------------
  27  32   5   3  | VB->NNP if Pos:TO@[-1]
  13  15   2   1  | NN->NNP if Pos:,@[1]
   8   9   1   5  | JJ->NN if Pos:IN@[-3,-2,-1]
   8   8   0   0  | IN->NN if Word:hsr@[-1]
   6   6   0   0  | RB->NNP if Pos:,@[-1]
   5   5   0   0  | PRP->NN if Pos:VBP@[1]
   4   4   0   0  | IN->NNP if Pos:,@[-1]
   4   4   0   0  | NN->NNP if Pos:,@[1]
   4   4   0   1  | JJ->NNP if Pos:TO@[-3,-2,-1]
   4   4   0   0  | NN->NNP if Word:stackroute@[-2]


In [181]:
d.respond('book a cab to Wipro, Electronic city' )

['source'] 1


'Please provide all the necessary details'

In [182]:
d.respond('Shreevastav')

[] 0


''

In [183]:
d.respond('book')

[] 0


'Looking for a cab from shreevastav to wipro , electronic city\nDistance between two points exceeds 100 miles'